# Week 10
# Data Loading and Storage

Accessing data is a necessary first step for most data science projects. From this chapter we will learn:
- Reading and writing data in text format (.txt, .csv, .json)
- Reading data from webpages (web scrapping)
- Reading and writing data in binary format (.pickle, .feather, .h5)
- Interacting with databases

Reading:
- Textbook, Chapter 6

## I. Reading and Writing Data in Text Format

### 1. csv file

In [ ]:
# Let's create a data frame first
import numpy as np
import pandas as pd

values = np.array([
    [100, 80, 95, 'A'],
    [55, 60, 45, 'F'],
    [70, 75, 90, 'A'],
    [75, 70, 60, 'D'],
    [60, 73, 75, 'C'],
    [72, 63, -1, 'NA']
])
df = pd.DataFrame(values,
                   columns=['Midterm', 'Project', 'Final', 'LetterGrade'],
                   index=['Alex', 'Bob', 'Chris', 'Doug', 'Eva', "Frank"])
df

In [ ]:
# Write to a csv file using .to_csv()
import os
print('Does path "Data/temp/" exist?', os.path.exists("Data/temp/"))

if not os.path.exists("Data/temp/"):
    os.mkdir("Data/temp")
    print('File path "Data/temp/" created.')

df.to_csv("Data/temp/grades.csv")

In [ ]:
# Load the csv file
df2 = pd.read_csv("Data/temp/grades.csv", sep=",")
df2

In [ ]:
# Load only the first 3 rows
df3 = pd.read_csv("Data/temp/grades.csv", sep=',', nrows=3)
df3

In [ ]:
# Load the file, skipping row 2 and 4
df4 = pd.read_csv("Data/temp/grades.csv", sep=',', skiprows=[2, 4])
df4

In [ ]:
# Remove column headers from the csv file, then load it
# df5 = pd.read_csv("Data/temp/grades.csv", header=None, names=['Name', 'Midterm', 'Project', 'Final', 'LetterGrade'])
df5 = pd.read_csv("Data/temp/grades.csv", sep=',', names=[1, 2, 3, 4, 5])
df5

In [ ]:
# Set first column as index
df6 = pd.read_csv("Data/temp/grades.csv", sep=',', index_col=0)
df6

In [ ]:
# Identify -1 as NaN
df7 = pd.read_csv("Data/temp/grades.csv", delimiter=',', na_values=[-1])
df7

### 2. Load txt file with values separated by spaces

In [ ]:
with open("Data/temp/values.txt", 'w') as file:
    file.write("Index Category     Value\n")
    file.write("1            A      2.92\n")
    file.write("2            B     12.14\n")
    file.write("3            C    123.56\n")

In [ ]:
# Although read_csv() is still applicable, setting delimiter to a single space will create errors
df = pd.read_csv("Data/temp/values.txt", sep=" ")
df

In [ ]:
df = pd.read_csv("Data/temp/values.txt", sep="\s+")
df

### 3. Load JSON files

**JavaScript Object Notation (JSON)** is a popular file format to storing unstructured data because it is easy for both human and computer to understand.
- Its structure is very similar to Python dictionary
- Load a json file with json.loads()
- Writes to a json file with json.dump()

In [ ]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [ ]:
import json
result = json.loads(obj)
result

In [ ]:
?result

In [ ]:
asjson = json.dumps(result)

In [ ]:
# Use json.dump(object, file) to write the content to file
with open("Data/temp/People.json", 'w') as file:
    json.dump(result, file)

In [ ]:
# Load from People.json
with open("Data/temp/People.json", "r") as file:
    people = json.load(file)
people

In [ ]:
# Load the content as a data frame
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])
siblings

## II. Web Scrapping
When performing data science tasks, it's common to want to use data found on the internet. You'll usually be able to access the data in csv format, or via an Application Programming Interface (API). However, there are times when the data you want can only be accessed as part of a web page. In cases like this, you'll want to use a technique called **web scraping** to get the data from the web page into a format you can work with in your analysis.

In [ ]:
# Download a webpage
import requests
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page #2** status code usually means successful download

In [ ]:
# Show what is downloaded
print(page.content)

We will use **beautifulsoup** library to extract useful information from the html script.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

In [ ]:
# using the children attribute to select all the top-level tags
list(soup.children)

In [ ]:
# type of each children
print([type(item) for item in list(soup.children)])

In [ ]:
# select the html tag and its children by taking the third item in the list:
html = list(soup.children)[2]
print(html)

In [ ]:
print('\n'.join([str(idx) + ':\n' + str(item) \
                 for idx, item in enumerate(list(html.children))]))

In [ ]:
len(list(html.children))

In [ ]:
print([type(item) for item in list(html.children)])

In [ ]:
body = list(html.children)[3]
print(body)

In [ ]:
print(list(body.children))

In [ ]:
p = list(body.children)[1]
print(p)

In [ ]:
p.get_text()

#### FInding all instances of a tag at once

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

In [ ]:
soup.find_all('p')[0].get_text()

In [ ]:
# Find the first instance of a tag
soup.find('p')

#### Searching for tags by class and id

In [ ]:
# Let's look at another webpage with classes and id's
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

In [ ]:
# Find all tags of a class
soup.find_all(class_="outer-text")

In [ ]:
soup.find_all(id="first")

In [ ]:
soup.find_all('p')

#### Downloading the weather data
1. Open the [weather forecast page](https://forecast.weather.gov/MapClick.php?lat=40.7146&lon=-74.0071#.Xbc5aXVKhhE)
2. Display the source code (On Chrome use "Developer Tools")
3. Identify the item containing data (On Chrome right click the values and select "Inspect")

In [ ]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=40.7146&lon=-74.0071#.Xbc5aXVKhhE")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
# print(len(seven_day))
# print(seven_day)
forecast_items = seven_day.find_all(class_="tombstone-container")
# print(len(forecast_items))
# print(forecast_items)
tonight = forecast_items[0]
print(tonight.prettify())

In [ ]:
# Find today's weather




In [ ]:
# Find weather forecast for the week
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

In [ ]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

In [ ]:
# Load the weather data as a data frame
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

In [ ]:
weather.dtypes

In [ ]:
# extract numeric temperature
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html
weather["temp_num"] = temp_nums.astype('int')
weather

In [ ]:
# Identify day temperature from night temperature
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

In [ ]:
weather[is_night]

In [ ]:
# Get new headlines from New York Times?
# Get current stock prices?
# Monitor alarms?;
# Download files?